In [ ]:
include("../load.jl")
include("../tools/plots.jl")

In [ ]:
# BBF 1
m = Model(with_optimizer(CPLEX_SILENT))
@variable(m, -1 <= x <= 1)
@variable(m, -1 <= y <= 1)
gm = GlobalModel(model = m)
add_nonlinear_constraint(gm, :((x,y) -> x^2*(0.2-y) + y))
bbf = gm.bbfs[1]
bbf.constraint

In [ ]:
# BBF 2 
gm = minlp(true)
bbf = gm.bbfs[1]
bbf.constraint

In [ ]:
# BBF 3 
gm = minlp(true)
bbf = gm.bbfs[2]
bbf.constraint

In [ ]:
# BBF 1 as an objective
m = Model(with_optimizer(CPLEX_SILENT))
@variable(m, -1 <= x <= 1)
@variable(m, -1 <= y <= 1)
@variable(m, -1 <= obj <=1)
gm = GlobalModel(model = m)
add_nonlinear_constraint(gm, :((x,y, obj) -> obj - (x^2*(0.2-y) + y)))
bbf = gm.bbfs[1]
bbf.constraint

In [ ]:
# Uniform sampling
sample_and_eval!(bbf, lh_iterations=10)
plot_2d(bbf)

In [ ]:
# KNN sampling
sample_and_eval!(bbf)
plot_2d(bbf)

In [ ]:
# Vanilla training
learn_constraint!(bbf)

In [ ]:
# Plotting predictions
plot_2d_predictions(bbf)

In [ ]:
bbf.accuracies

In [ ]:
show_trees(bbf)